In [4]:
import sys
import pickle
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# -----------------------------
# PATH SETUP
# -----------------------------
HERE = Path.cwd()

ROOT = HERE.parent
SRC_DIR = ROOT / "src"
if not SRC_DIR.exists():
    ROOT = HERE
    SRC_DIR = ROOT / "src"

DATA_DIR = ROOT / "data"
MODEL_DIR = ROOT / "models"

sys.path.insert(0, str(SRC_DIR))

from feature_engineering import prepare_intent_features, prepare_verification_features

# -----------------------------
# Helper to load model
# -----------------------------
def load_model(path):
    with open(path, "rb") as f:
        return pickle.load(f)

# -----------------------------
# Helper to evaluate
# -----------------------------
def evaluate(model, X, y, name="MODEL"):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )

    y_pred = model.predict(X_test)

    print(f"\n===== {name} =====")
    print("Accuracy:", round(accuracy_score(y_test, y_pred), 3))
    print(classification_report(y_test, y_pred))

# -----------------------------
# 1) INTENT MODEL ACCURACY
# -----------------------------
intent_csv = DATA_DIR / "intent_train.csv"
intent_model_path = MODEL_DIR / "intent_model.pkl"

X_intent, y_intent = prepare_intent_features(str(intent_csv))
intent_model = load_model(intent_model_path)

evaluate(intent_model, X_intent, y_intent, name="INTENT MODEL")

# -----------------------------
# 2) VERIFICATION MODEL ACCURACY
# -----------------------------
verify_csv = DATA_DIR / "fake_train.csv"
verify_model_path = MODEL_DIR / "verification_model.pkl"

X_ver, y_ver = prepare_verification_features(str(verify_csv))
verification_model = load_model(verify_model_path)

evaluate(verification_model, X_ver, y_ver, name="VERIFICATION MODEL")



===== INTENT MODEL =====
Accuracy: 0.933
              precision    recall  f1-score   support

           0       1.00      0.87      0.93        63
           1       0.88      1.00      0.93        57

    accuracy                           0.93       120
   macro avg       0.94      0.94      0.93       120
weighted avg       0.94      0.93      0.93       120


===== VERIFICATION MODEL =====
Accuracy: 0.925
              precision    recall  f1-score   support

           0       0.95      0.96      0.95        94
           1       0.84      0.81      0.82        26

    accuracy                           0.93       120
   macro avg       0.89      0.88      0.89       120
weighted avg       0.92      0.93      0.92       120

